# Chapter 4: Minimax Tree Search

In this chapter, you’ll learn how minimax algorithms work. You'll use it to play the Last Coin Standing game (a.k.a. the coin game) that we developed in Chapter 1. The algorithm exhausts all possibilities in the coin game. The minimax agent plays perfectly: it always wins when it pays second. 

The minimax algorithm is a decision rule in artificial intelligence and game theory. The algorithm assumes that each player in the game makes the best possible decisions at each step. Further, each player knows that other players make fully rational decisions as well. In simple games such as the coin game or Tic Tac Toe, the algorithm exhausts all possible game paths and solves the game. This means the minimax algorithm provides the strongest possible for these games and no other algorithms can beat it. 

In more complicated games such as Connect Four or Chess, the minimax algorithm cannot exhaust all possibilities in a short amount of time. However, modifications on the minimax algorithms such as depth pruning and alpha-beta pruning (which we'll study in the next couple of chapters) make strong game players. In fact, the algorithms used by Deep Blue to beat World Chess Champion Gary Kasparov in 1997 is based on minimax with alpha-beta pruning and powerful position evaluation functions. 

After this chapter, you'll understand the basic logi behind the minimax algorithm and can design game strategies for any game based on the algorithm. 

***
$\mathbf{\text{Create a subfolder for files in Chapter 4}}$<br>
***
We'll put all files in Chapter 4 in a subfolder /files/ch04. Run the code in the cell below to create the subfolder.

***

In [1]:
import os

os.makedirs("files/ch04", exist_ok=True)

# 1. Introduction to the Minimax Algorithm
We'll introduce the minimax algorithm in the section.

## 1.1. What is the Minimax Algorithm?
The minimax algorithm is a decision rule in artificial intelligence and game theory. It's also called the MinMax algorithm. 

In a nutshell, the algorithm assumes that: 
 
*	Each player in the game makes the best possible decisions at each step;
*	Further, each player knows that other players make fully rational decisions as well;
*	Each player knows that other players know that he/she makes the best possible decisions;
*   and so on...

In a two-player game, each player makes decisions to maximize his/her own expected payoff and to minimize the opponent's payoffs. Hence the name minimax. 

## 1.2. The Minimax Algorithm with Backward Induction
The solution in a minimax algorithm is achieved through backward induction. It starts with the terminal state of the game. In the coin game, this is when the last coin is taken by a player; in Tic Tac Toe, the terminal state is when the game is tied or when one player has connected three pieces in a row. We find out the payoff to each player in the terminal state. In the second to last stage of the game, the player looks one step ahead and makes the best decision for himself/herself, anticipating that the opponent makes the best decision in the next step, and so on.

Let's use the coin game as the example. If both players make the best decisions, the game has seven rounds and 14 steps (in each round, player 1 makes a move and player 2 makes a move). In stage 7, three coins are left in the pile and player 1 has to decide how many coins to take: 1 or 2? At this point, player 1 knows that she will lose for sure: player 2 will take the remaining one or two coins in the pile and win the game. In stage 7, six coins are left in the pile and player 1 has to decide how many coins to take: 1 or 2? Player 1 knows that if she takes 1, player will take 2; if she takes 2, player 2 will take 1. Player 2 will leave three coins in the pile no matter what... In stage 1, 21 coins are in the pile and player 1 has to decide how many coins to take: 1 or 2? Player 1 knows that if she takes 1, player will take 2; if she takes 2, player 2 will take 1. Player 2 will leave 18 coins in the pile no matter what... 

Since the total number of possible scenarios in a Tic Tac Toe game is small, the minimax algorithm can exhaust all scenarios in a short amount of time and find the best solution for each player in every stage of the game. 

However, for more complicated games such as Chess and Go, the total number of possible scenarios in a game is astronomical. It's impossible for the minimax algorithm to pick the best move in a reasonable amount of time. We'll discuss how to mitigate this concern in later chapters.

# 2. The Minimax Algorithm in the Coin Game

We'll use the self-made coin game environment from Chapter 1. Specifically, the module is saved as *coin_env.py* in the folder *utils* in this GitHub repository. Download the file *coin_env.py* and save it in the folder /Desktop/ai/utils/ on your computer.

First, let's define a couple of functions that the algorithm will use. 

## 2.1. The minimax() Function
We'll define a minimax() function for the player who is about to make a move. The function applies to both player 1 and player 2. The function tells the player what's the best next move, anticipating that the opponent will make the best decision in the next stage as well, and so on. 

In [2]:
from copy import deepcopy
from random import choice

def minimax(env):
    # create a list to store winning moves
    wins=[]
    # iterate through all possible next moves
    for m in env.validinputs:
        # make a hypothetical move and see what happens
        env_copy=deepcopy(env)
        new_state, reward, done, info = env_copy.step(m) 
        # If wins right away with move m, take it.
        if done and reward==1:
            return m 
        # See what's the best response from the opponent
        opponent_payoff=maximized_payoff(env_copy, reward, done)  
        # Opponent's payoff is the opposite of your payoff
        my_payoff=-opponent_payoff 
        if my_payoff==1:
            wins.append(m)
    # pick winning moves if there is any        
    if len(wins)>0:
        return choice(wins)
    # otherwise randomly pick
    return choice(env.validinputs)

To search for the best move, the player iterates through all possible next moves. Note that we need to use deepcopy here. In Python, when you make a regular copy of an object, you just create a link to the original object. When you make changes to the copy, the original is changed as well. To avoid this, we need to use deepcopy. Many Python beginners make mistakes on this and couldn't figure out what's wrong with their code.

If a player finds a move that allows her to win the game right away, the player stops searching and take the move. Otherwise, the player will see what's the best outcome for the opponent in the next stage, knowing full well that the opponent will make the best decision to maximize the opponent's payoff. Since it's a zero-sum game, the opponent's payoff is the opposite of the current player's payoff. The player will pick winning moves if there is one; otherwise, the player randomly picks a move. 

Here, we use the *maximized_payoff(env_copy, reward, done)* function to find the best payoff for the opponent in the next stage. Let's define that function next. 

***
$\mathbf{\text{Deepcopy in Python}}$<br>
***
In Python, when you make a regular copy of an object, you just create a link to the original object. When you make changes on the copy, the original is changed as well. To avoid this, we need to use deepcopy when we make hypothetical game moves. Many Python beginners make mistakes on this and couldn't figure out what's wrong with their code.

***

## 2.2. The *maximized_payoff()* Function 
Next, we'll define *maximized_payoff(env, reward, done)* function. This function produces the best possible outcome for the next player in the next stage of the game. Note this function applies to any player in any stage of the game so we don't need to define one function for player 1 and another function for player 2.

In [3]:
def maximized_payoff(env, reward, done):
    # if the game has ended after the previous player's move
    if done:
        return -1
    # Otherwise, search for action to maximize payoff
    best_payoff=-2
    # iterate through all possible moves
    for m in env.validinputs:
        env_copy=deepcopy(env)
        new_state,reward,done,info=env_copy.step(m)  
        # If I make this move, what's the opponent's response
        opponent_payoff=maximized_payoff(env_copy, reward, done)
        # Opponent's payoff is the opposite of your payoff
        my_payoff=-opponent_payoff 
        # update your best payoff 
        if my_payoff>best_payoff:        
            best_payoff=my_payoff
    return best_payoff

If the game has ended after the previous player's move, the function calculates the payoff to the next player based on the game outcome. In this case, it means the previous player has won the game, so the payoff to the current player is -1. If the game has not ended, the player searches for the best action by iterating throug all possible next moves, knowing full well that the opponent will take the best action in the next stage as well.

Note here that we have used the *maximized_payoff()* function in the *maximized_payoff()* function itself. This creates an nested loop. The function keeps on searching to the next level until the game ends. The process exhausts all game scenarios in the coin game.

***
$\mathbf{\text{Use A Function in the Function Itself}}$<br>
***
In Python, you can put a function in the function itself. This creates an infinite loop. The function keeps on going to the next stage until a certain condition is met. In the case of the maximized_payoff() function above. The infinite loop stops until the coin game is finished (that is, there is no coin left in the pile and a player has won). The process exhausts all game scenarios. 
***

## 2.3. Play Against the Minimax Algorithm 
Next, you'll play a game against the minimax algorithm. We'll let the minimax algorithm move second and see if it wins the game 100% of the time. 

In [4]:
from utils.coin_env import coin_game

# Initiate the game environment
env=coin_game()
state=env.reset()   
print("enter a move in the form of 1 or 2")
# Play a full game
while True:
    print(f"there are {state} coins in the pile")   
    action = input("Player 1, what's your choice: 1 or 2?")
    print(f"Player 1 has chosen action={action}")    
    state, reward, done, info = env.step(action)
    if done:
        print(f"there are {state} coins in the pile")
        print(f"Player 1 has won!") 
        break
    print(f"there are {state} coins in the pile") 
    action = minimax(env)
    print(f"Player 2 has chosen action={action}")  
    state, reward, done, info = env.step(action)
    if done:
        print(f"there are {state} coins in the pile")
        print(f"Player 2 has won!") 
        break

enter a move in the form of 1 or 2
there are 21 coins in the pile
Player 1, what's your choice: 1 or 2?2
Player 1 has chosen action=2
there are 19 coins in the pile
Player 2 has chosen action=1
there are 18 coins in the pile
Player 1, what's your choice: 1 or 2?1
Player 1 has chosen action=1
there are 17 coins in the pile
Player 2 has chosen action=2
there are 15 coins in the pile
Player 1, what's your choice: 1 or 2?1
Player 1 has chosen action=1
there are 14 coins in the pile
Player 2 has chosen action=2
there are 12 coins in the pile
Player 1, what's your choice: 1 or 2?2
Player 1 has chosen action=2
there are 10 coins in the pile
Player 2 has chosen action=1
there are 9 coins in the pile
Player 1, what's your choice: 1 or 2?2
Player 1 has chosen action=2
there are 7 coins in the pile
Player 2 has chosen action=1
there are 6 coins in the pile
Player 1, what's your choice: 1 or 2?1
Player 1 has chosen action=1
there are 5 coins in the pile
Player 2 has chosen action=2
there are 3 coi

The minimax algorithm wins the game. It makes sure the number of coins in the pile is a multiple of 3: 18, 15, 12, 9, 6, and then 3.

# 3. Test the Efficacy of the Minimax Algorithm
Next, we’ll test how often the Minimax Algorithm wins against the rule-based AI game strategy that we developed in Chapter 1. We'll first define a few functions in the ch04util module. 

## 3.1. The test_coin_game() Function

Download the file ch04util.py from the book's GitHub repository and save it in /Desktop/ai/utils/ on your computer. The file acts as a local module with a few functions in it.

The first two functions are maximized_payoff() and minimax() and they are exactly the same as we have defined in the last section. The third function is rule_based_AI() function, which is defined as follows:

In [5]:
def rule_based_AI(env):
    state=int(env.state)
    if state%3 != 0:
        move = state%3
    else:
        move = choice([1,2])
    return move

The function is similar to the one we have defined in Chapter 1, except that it doesn't take any arguments. It generates the variable *state* based on the current game state in the coin game environment. 

The fourth function is the random_player() function. The function randomly picks a legal move, and is defined as follows:

In [6]:
def random_player(env):
    return choice(env.validinputs)

The last function is the test_coin_game() function. It test a coin game between two game strategies. The function takes two arguments: player1 and player2, both are function names. For example, if we put minimax and random_player as the two arguments and in that order, the test_coin_game() function simulates a game between the minimax strategy and a random-move strategy, where the minimax strategy moves first. The test_coin_game() function returns the game outcome: 1 if player 1 wins and -1 if player 2 wins. 

In [7]:
def test_coin_game(env,player1,player2):
    state=env.reset()   
    while True:
        action=player1(env)   
        nstate,reward,done,info=env.step(action)
        if done:
            return 1 
        action=player2(env)   
        nstate,reward,done,info=env.step(action)
        if done:
            return -1

## 3.2. The Minimax Agent versus Random Moves

Next, we'll see how good the minimax algorithm is when it plays against random moves. To speed up testing, we created a new game environment for the coin game, which is in the file coin_simple_env.py in the folder /utils/. It's the same as coin_env.py except that we omit the graphical game window functionality. As a result, you cannot use the render() method in the simplified game environment. 

We simulate 100 games and let the minimax agent move first.

In [8]:
from utils.ch04util import minimax, random_player, test_coin_game
from utils.coin_env import coin_game

env=coin_game()
results=[]
for i in range(100):
    # minimax moves second 
    result=test_coin_game(env,minimax,random_player)
    # record game outcome
    results.append(result)

# count how many times minimax has won
wins=results.count(1)
print(f"the minimax algorithm won {wins} games")
# count how many times AI player has lost
losses=results.count(-1)
print(f"the minimax algorithm lost {losses} games")
# count tie games
ties=results.count(0)
print(f"the game has tied {ties} games")  

the minimax algorithm won 99 games
the minimax algorithm lost 1 games
the game has tied 0 games


Results above show that the minimax agent has won 99 games out of 100. 

Since we know that if both players use perfect strategies, the second player always wins, we'll see what happens if the minimax agent moves second. 

In [9]:
results=[]
for i in range(100):
    # minimax moves second 
    result=test_coin_game(random_player,minimax)
    # record negative game outcome
    results.append(-result)

# count how many times minimax has won
wins=results.count(1)
print(f"the minimax algorithm won {wins} games")
# count how many times AI player has lost
losses=results.count(-1)
print(f"the minimax algorithm lost {losses} games")
# count tie games
ties=results.count(0)
print(f"the game has tied {ties} games")  

the minimax algorithm won 100 games
the minimax algorithm lost 0 games
the game has tied 0 games


We record the negative of the game outcome so that a value of 1 in the list *results* indicates that the minimax agent has won. The output from the above cell shows that the minimax agent has won all 100 games. 

## 3.3. The Minimax Agent versus Rule-Based AI

Next, we'll see how good the minimax algorithm is when it plays against the rule-based AI agent that we developed in Chapter 1. Below, we simulate 100 games and let the minimax agent move first.

In [10]:
from utils.ch04util import rule_based_AI

results=[]
for i in range(100):
    # minimax moves second 
    result=test_coin_game(env,minimax,rule_based_AI)
    # record game outcome
    results.append(result)

# count how many times minimax has won
wins=results.count(1)
print(f"the minimax algorithm won {wins} games")
# count how many times AI player has lost
losses=results.count(-1)
print(f"the minimax algorithm lost {losses} games")
# count tie games
ties=results.count(0)
print(f"the game has tied {ties} games")  

the minimax algorithm won 0 games
the minimax algorithm lost 100 games
the game has tied 0 games


Results above show that the minimax agent has lost all 100 games. Since we know that if both players use perfect strategies, the second player always wins, we'll see what happens if the minimax agent moves second. 

In [11]:
results=[]
for i in range(100):
    # minimax moves second 
    result=test_coin_game(env,rule_based_AI,minimax)
    # record negative game outcome
    results.append(-result)

# count how many times minimax has won
wins=results.count(1)
print(f"the minimax algorithm won {wins} games")
# count how many times AI player has lost
losses=results.count(-1)
print(f"the minimax algorithm lost {losses} games")
# count tie games
ties=results.count(0)
print(f"the game has tied {ties} games")  

the minimax algorithm won 100 games
the minimax algorithm lost 0 games
the game has tied 0 games


The output from the above cell shows that the minimax agent has won all 100 games. 

Taken together, our results show that the minimax agent plays the game perfectly in the sense that whenever it plays second, it wins 100% of the time. 